<a href="https://colab.research.google.com/github/gogumalatte/yolov8-detection-webapp/blob/main/YOLOv8_%EA%B8%B0%EB%B0%98_%EC%8B%A4%EC%8B%9C%EA%B0%84_%EA%B0%9D%EC%B2%B4_%EC%9D%B8%EC%8B%9D_%EC%9B%B9_%EC%84%9C%EB%B9%84%EC%8A%A4_%EA%B5%AC%EC%B6%95_%EA%B0%80%EC%9D%B4%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 객체 인식을 위한 YOLOv8 (Ultralytics)와, 실시간 웹 앱 구성을 위한 Flask/Socket 관련 패키지를 설치합니다.

In [9]:
# 필수 라이브러리 설치
!pip install flask flask-cors ultralytics pyngrok
!pip install watchdog  # 파일 변경 감지용
!pip install ultralytics # YOLOv8 설치
from ultralytics import YOLO
model = YOLO('yolov8n.pt')  # 작은 모델 사용

# 2. Ngrok을 통해 로컬 서버를 외부에서 접근할 수 있도록 인증 토큰을 설정합니다.

In [10]:
!ngrok authtoken 2wQfPVBCGGvd0mYA96vUepa8w7g_4TsNZesSbc6nvXmV38T3T

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


# 3. Ngrok이 정상적으로 설치되었는지 버전을 확인합니다.

In [11]:
!ngrok --version

ngrok version 3.22.1
pyngrok version 7.2.8


# 4. React 프론트엔드 정적 파일을 저장할 www/js 디렉토리를 생성합니다.

In [12]:
# 필요한 디렉토리 생성
!mkdir -p www/js

# 5. 클라이언트와 서버 간의 실시간 비디오 전송 및 결과 공유를 위한 기본 서버 구성을 위한 백엔드 코드를 작성합니다.

In [13]:
%%writefile app.py
from flask import Flask, request, jsonify, render_template
from flask_cors import CORS
import os
import time
import base64
import numpy as np
import cv2
from ultralytics import YOLO
import json

# Flask 앱 초기화
app = Flask(__name__, template_folder='./www', static_folder='./www', static_url_path='/')
CORS(app)  # CORS 설정 추가

# YOLOv8 모델 로드
model = YOLO('yolov8n.pt')

# 임시 이미지 저장 경로
UPLOAD_FOLDER = './uploads'
if not os.path.exists(UPLOAD_FOLDER):
    os.makedirs(UPLOAD_FOLDER)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/api/detect', methods=['POST'])
def detect_objects():
    if 'image' not in request.json:
        return jsonify({'error': 'No image data provided'}), 400

    try:
        # 이미지 데이터 파싱
        img_data = request.json['image']
        # Base64 데이터에서 헤더 제거
        if ',' in img_data:
            img_data = img_data.split(',')[1]

        # Base64 디코딩
        img_bytes = base64.b64decode(img_data)
        nparr = np.frombuffer(img_bytes, np.uint8)
        image = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

        # 이미지 저장 (디버깅용)
        timestamp = int(time.time())
        img_path = f"{UPLOAD_FOLDER}/image_{timestamp}.jpg"
        cv2.imwrite(img_path, image)

        # YOLOv8로 객체 감지
        results = model(image)

        # 결과 처리
        detections = []
        for r in results:
            boxes = r.boxes
            for box in boxes:
                x1, y1, x2, y2 = box.xyxy[0].tolist()
                conf = float(box.conf[0])
                cls = int(box.cls[0])
                name = model.names[cls]

                detections.append({
                    'bbox': [x1, y1, x2, y2],
                    'confidence': conf,
                    'class': cls,
                    'name': name
                })

        return jsonify({
            'success': True,
            'detections': detections
        })

    except Exception as e:
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=3000, debug=True)

Overwriting app.py


# 6. 사용자와 상호 작용을 할 프론트엔드 코드를 작성합니다.
## HTML + JavaScript + React

In [14]:
%%writefile www/index.html
<!DOCTYPE html>
<html lang="ko">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>YOLOv8 객체 인식</title>
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/bootstrap/5.2.3/css/bootstrap.min.css">
    <style>
        body {
            font-family: Arial, sans-serif;
            padding: 20px;
            max-width: 600px;
            margin: 0 auto;
        }
        .camera-container {
            position: relative;
            width: 100%;
            max-width: 500px;
            margin: 0 auto;
        }
        #videoElement {
            width: 100%;
            background-color: #666;
        }
        #canvasElement {
            position: absolute;
            top: 0;
            left: 0;
            width: 100%;
            height: 100%;
        }
        .controls {
            margin: 20px 0;
            display: flex;
            justify-content: space-between;
        }
        .detection-list {
            margin-top: 20px;
            padding: 10px;
            background-color: #f8f9fa;
            border-radius: 5px;
            max-height: 200px;
            overflow-y: auto;
        }
        .detection-item {
            display: flex;
            justify-content: space-between;
            padding: 5px 0;
            border-bottom: 1px solid #dee2e6;
        }
        .spinner-border {
            display: none;
        }
    </style>
</head>
<body>
    <div id="app"></div>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/react/18.2.0/umd/react.production.min.js"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/react-dom/18.2.0/umd/react-dom.production.min.js"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/babel-standalone/7.21.2/babel.min.js"></script>
    <script type="text/babel" src="js/app.js"></script>
</body>
</html>

Overwriting www/index.html


In [15]:
%%writefile www/js/app.js
const { useState, useEffect, useRef } = React;

const App = () => {
    const [cameraMode, setCameraMode] = useState('environment'); // 'environment' (후면) 또는 'user' (전면)
    const [detections, setDetections] = useState([]);
    const [isProcessing, setIsProcessing] = useState(false);
    const [errorMessage, setErrorMessage] = useState('');
    const videoRef = useRef(null);
    const canvasRef = useRef(null);
    const streamRef = useRef(null);
    const timerRef = useRef(null);

    useEffect(() => {
        startCamera();
        return () => {
            stopCamera();
            if (timerRef.current) {
                clearInterval(timerRef.current);
            }
        };
    }, [cameraMode]);

    const startCamera = async () => {
        try {
            if (streamRef.current) {
                stopCamera();
            }

            const constraints = {
                video: {
                    facingMode: cameraMode
                }
            };

            const stream = await navigator.mediaDevices.getUserMedia(constraints);
            streamRef.current = stream;

            if (videoRef.current) {
                videoRef.current.srcObject = stream;
                videoRef.current.play();

                // 비디오 크기에 맞게 캔버스 설정
                videoRef.current.onloadedmetadata = () => {
                    if (canvasRef.current) {
                        canvasRef.current.width = videoRef.current.videoWidth;
                        canvasRef.current.height = videoRef.current.videoHeight;
                    }

                    // 주기적으로 객체 감지 실행
                    timerRef.current = setInterval(() => {
                        if (!isProcessing) {
                            detectObjects();
                        }
                    }, 10000); // 10초마다
                };
            }
        } catch (err) {
            setErrorMessage(`카메라 접근 오류: ${err.message}`);
            console.error('카메라 접근 오류:', err);
        }
    };

    const stopCamera = () => {
        if (streamRef.current) {
            streamRef.current.getTracks().forEach(track => track.stop());
            streamRef.current = null;
        }
    };

    const switchCamera = () => {
        setCameraMode(prev => prev === 'environment' ? 'user' : 'environment');
    };

    const captureImage = () => {
        if (!videoRef.current || !canvasRef.current) return null;

        const canvas = document.createElement('canvas');
        canvas.width = videoRef.current.videoWidth;
        canvas.height = videoRef.current.videoHeight;

        const ctx = canvas.getContext('2d');
        ctx.drawImage(videoRef.current, 0, 0, canvas.width, canvas.height);

        return canvas.toDataURL('image/jpeg');
    };

    const detectObjects = async () => {
        if (!videoRef.current || !canvasRef.current) return;

        try {
            setIsProcessing(true);
            const imageData = captureImage();

            if (!imageData) {
                console.error('이미지 캡처 실패');
                return;
            }

            const response = await fetch('/api/detect', {
                method: 'POST',
                headers: {
                    'Content-Type': 'application/json',
                },
                body: JSON.stringify({ image: imageData }),
            });

            const result = await response.json();

            if (result.success) {
                console.log('감지 결과:', result.detections);
                setDetections(result.detections);
                drawDetections(result.detections);
            } else {
                console.error('객체 감지 오류:', result.error);
                setErrorMessage(`객체 감지 오류: ${result.error}`);
            }
        } catch (err) {
            console.error('API 요청 오류:', err);
            setErrorMessage(`API 요청 오류: ${err.message}`);
        } finally {
            setIsProcessing(false);
        }
    };

    const drawDetections = (detectionResults) => {
        if (!canvasRef.current || !videoRef.current) return;

        const canvas = canvasRef.current;
        const ctx = canvas.getContext('2d');

        // 캔버스 초기화
        ctx.clearRect(0, 0, canvas.width, canvas.height);

        // 감지된 객체 그리기
        const scaleX = canvas.width / videoRef.current.videoWidth;
        const scaleY = canvas.height / videoRef.current.videoHeight;

        detectionResults.forEach(detection => {
            const [x1, y1, x2, y2] = detection.bbox;
            const scaledX1 = x1 * scaleX;
            const scaledY1 = y1 * scaleY;
            const scaledWidth = (x2 - x1) * scaleX;
            const scaledHeight = (y2 - y1) * scaleY;

            // 경계 상자 그리기
            ctx.strokeStyle = '#00FF00';
            ctx.lineWidth = 2;
            ctx.strokeRect(scaledX1, scaledY1, scaledWidth, scaledHeight);

            // 텍스트 배경 그리기
            const label = `${detection.name} ${Math.round(detection.confidence * 100)}%`;
            const textWidth = ctx.measureText(label).width;
            ctx.fillStyle = 'rgba(0, 0, 0, 0.7)';
            ctx.fillRect(scaledX1, scaledY1 - 20, textWidth + 10, 20);

            // 텍스트 그리기
            ctx.fillStyle = '#FFFFFF';
            ctx.font = '16px Arial';
            ctx.fillText(label, scaledX1 + 5, scaledY1 - 5);
        });
    };

    const renderDetectionList = () => {
        if (detections.length === 0) {
            return <p>감지된 객체가 없습니다.</p>;
        }

        // 클래스별로 그룹화
        const groupedDetections = {};
        detections.forEach(detection => {
            const name = detection.name;
            if (!groupedDetections[name]) {
                groupedDetections[name] = 0;
            }
            groupedDetections[name]++;
        });

        return (
            <div>
                <h5>감지된 객체</h5>
                {Object.entries(groupedDetections).map(([name, count], index) => (
                    <div key={index} className="detection-item">
                        <span>{name}</span>
                        <span className="badge bg-primary">{count}</span>
                    </div>
                ))}
                <p className="mt-2">총 {detections.length}개 객체 감지됨</p>
            </div>
        );
    };

    return (
        <div className="container">
            <h1 className="text-center mb-4">YOLOv8 객체 인식</h1>

            {errorMessage && (
                <div className="alert alert-danger" role="alert">
                    {errorMessage}
                </div>
            )}

            <div className="camera-container">
                <video ref={videoRef} id="videoElement" autoPlay playsInline></video>
                <canvas ref={canvasRef} id="canvasElement"></canvas>
            </div>

            <div className="controls mt-3">
                <button
                    className="btn btn-primary"
                    onClick={switchCamera}
                >
                    {cameraMode === 'environment' ? '전면 카메라로 전환' : '후면 카메라로 전환'}
                </button>

                <button
                    className="btn btn-success"
                    onClick={detectObjects}
                    disabled={isProcessing}
                >
                    객체 감지
                    {isProcessing && (
                        <span className="spinner-border spinner-border-sm ms-2" role="status" aria-hidden="true"></span>
                    )}
                </button>
            </div>

            <div className="detection-list mt-3">
                {renderDetectionList()}
            </div>
        </div>
    );
};

ReactDOM.render(<App />, document.getElementById('app'));

Overwriting www/js/app.js


# 7. 한 번에 위 서버를 실행하고 접근 URL을 여는 실행 스크립트를 작성합니다.

In [16]:
%%writefile run_server.py
import subprocess
import time
from pyngrok import ngrok

# Flask 서버 시작
server_process = subprocess.Popen(["python", "app.py"])
print("Flask 서버가 시작되었습니다.")

# ngrok 터널 생성
http_tunnel = ngrok.connect(3000)
print(f"ngrok 터널이 생성되었습니다: {http_tunnel.public_url}")

try:
    # 앱이 계속 실행되도록 대기
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    # 종료 시 프로세스 정리
    server_process.terminate()
    ngrok.kill()

# 이 코드는 Colab이 계속 실행 중일 때만 작동합니다

Writing run_server.py


# 8. 프로그램에 필요한 디렉토리를 생성하고, 7단계에서 생성한 스크립트를 실행합니다. (서버를 실행)

In [17]:
# 필요한 디렉토리 생성
!mkdir -p uploads

# 서버 실행
!python run_server.py

Flask 서버가 시작되었습니다.
ngrok 터널이 생성되었습니다: https://4845-35-247-177-53.ngrok-free.app
 * Serving Flask app 'app'
 * Debug mode: on
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:3000
 * Running on http://172.28.0.12:3000
Press CTRL+C to quit
 * Restarting with watchdog (inotify)
 * Debugger is active!
 * Debugger PIN: 347-788-323
127.0.0.1 - - [18/May/2025 17:41:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/May/2025 17:41:08] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [18/May/2025 17:41:08] "GET /js/app.js HTTP/1.1" 200 -


0: 480x640 1 person, 1 chair, 1 couch, 1 remote, 43.9ms
Speed: 7.8ms preprocess, 43.9ms inference, 361.6ms postprocess per image at shape (1, 3, 480, 640)
127.0.0.1 - - [18/May/2025 17:41:24] "POST /api/detect HTTP/1.1" 200 -
0: 480x640 1 person, 1 chair, 1 couch, 1 remote, 55.0ms
Speed: 11.2ms preprocess, 55.0ms inference, 2.1ms postprocess per image at shape (1, 3, 480, 640)
127.0.0.1 - - [18/May/2025 17:41:24] "POST /api/detect HTTP/1.1" 200 -

In [18]:
!apt-get install tree

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 34 not upgraded.
Need to get 47.9 kB of archives.
After this operation, 116 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tree amd64 2.0.2-1 [47.9 kB]
Fetched 47.9 kB in 1s (54.5 kB/s)
Selecting previously unselected package tree.
(Reading database ... 126102 files and directories currently installed.)
Preparing to unpack .../tree_2.0.2-1_amd64.deb ...
Unpacking tree (2.0.2-1) ...
Setting up tree (2.0.2-1) ...
Processing triggers for man-db (2.10.2-1) ...


In [19]:
!tree

.
├── app.py
├── run_server.py
├── sample_data
│   ├── anscombe.json
│   ├── california_housing_test.csv
│   ├── california_housing_train.csv
│   ├── mnist_test.csv
│   ├── mnist_train_small.csv
│   └── README.md
├── uploads
│   ├── image_1747590075.jpg
│   ├── image_1747590081.jpg
│   ├── image_1747590091.jpg
│   ├── image_1747590101.jpg
│   ├── image_1747590135.jpg
│   ├── image_1747590145.jpg
│   ├── image_1747590154.jpg
│   ├── image_1747590156.jpg
│   ├── image_1747590166.jpg
│   ├── image_1747590176.jpg
│   ├── image_1747590186.jpg
│   ├── image_1747590196.jpg
│   ├── image_1747590206.jpg
│   ├── image_1747590216.jpg
│   └── image_1747590225.jpg
├── www
│   ├── index.html
│   └── js
│       └── app.js
└── yolov8n.pt

4 directories, 26 files
